In [1]:
#import all libraries
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

download the data and tranform it into a data frame

In [27]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source,'lxml')

In [28]:
table_can = soup.find('table')
cells = table_can.find_all('td')

postcode = []
borough = []
neighbourhood = []

for i in range(0, len(cells), 3):
    postcode.append(cells[i].text.strip())
    borough.append(cells[i+1].text.strip())
    neighbourhood.append(cells[i+2].text.strip())

In [29]:
df_pc = pd.DataFrame(data=[postcode, borough, neighbourhood]).transpose()
df_pc.columns = ['Postcode', 'Borough', 'Neighbourhood']
df_pc.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


ignore cells with a borough that is Not assigned

In [30]:
df_pc.drop(df_pc[df_pc['Borough'] == 'Not assigned'].index, inplace=True)

if a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [31]:
df_pc.loc[df_pc.Neighbourhood == "Not assigned", "Neighbourhood"] = df_pc.Borough

group the data according to the same postcode and borough

In [32]:
df_pc = df_pc.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df_pc.columns = ['Postcode', 'Borough', 'Neighbourhood']

to get the shape of the data frame

In [33]:
df_pc.shape

(103, 3)

In [34]:
df_pc.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


to set the index as postcode

In [42]:
df_pc.set_index('Postcode', drop =True, inplace = True)
df_pc.head()

,Borough,Neighbourhood
Postcode,,
M1B,Scarborough,"Rouge, Malvern"
M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


to download the geo-data

In [10]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.
client_8c6994d0c2af4c0e899abe707846c03e = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='leA883vXkrHNx7gQyObAnpk30dOgPfvXe_Zm-llhSeEJ',
    ibm_auth_endpoint="https://iam.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_8c6994d0c2af4c0e899abe707846c03e.get_object(Bucket='myproject1-donotdelete-pr-i3edivdwtdewmg',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_gc = pd.read_csv(body).set_index("Postal Code")
df_gc.head()

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


combine two data sets

In [43]:
Combined_data = df_pc.join(df_gc)
Combined_data

,Borough,Neighbourhood,Latitude,Longitude
Postcode,,,,
M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
M1G,Scarborough,Woburn,43.770992,-79.216917
M1H,Scarborough,Cedarbrae,43.773136,-79.239476
M1J,Scarborough,Scarborough Village,43.744734,-79.239476
M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476


In [44]:
import json # library to handle JSON files

In [51]:
from geopy.geocoders import Nominatim

In [52]:
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [53]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [54]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [55]:
import xml

In [58]:
CLIENT_ID = 'WWK0OPNLKLO4NYAXIJTHSVUZM2Y5MODZP43EZLKC5M2DHA2V' # your Foursquare ID
CLIENT_SECRET = 'HTAFAKDEZXZNNZ4KWS13C1OD5JBZYXLM5A1TLPXNG0GHXDBX' # your Foursquare Secret
VERSION = '20190504' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: WWK0OPNLKLO4NYAXIJTHSVUZM2Y5MODZP43EZLKC5M2DHA2V
CLIENT_SECRET:HTAFAKDEZXZNNZ4KWS13C1OD5JBZYXLM5A1TLPXNG0GHXDBX


In [65]:
df_toronto = Combined_data[Combined_data['Borough'].str.contains('Toronto')]

df_toronto.reset_index(inplace = True)
df_toronto

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
7,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049


create the map of the neighbourhoods of Toronto

In [71]:
import folium

In [73]:
address = 'Toronto, Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.19.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


In [74]:
# create map of Toronto using latitude and longitude values
map_neighbour = folium.Map(location=[latitude, longitude], zoom_start=11)

In [76]:
# add markers to map
for lat, lng, borough, neighbourhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3199cc',
        fill_opacity=0.3,
        parse_html=False).add_to(map_neighbour)  
    
map_neighbour

explore the first neighbourhood Lawrence Park